# The StatQuest Illustrated Guide to Neural Networks and AI
## Chapters 4 and 5 - ArgMax, SoftMax, and Cross Entropy!!!

Copyright 2024, Joshua Starmer

---- 

In this tutorial, we will use **[PyTorch](https://pytorch.org/) + [Lightning](https://www.lightning.ai/)** to create and optimize a simple neural network with multiple inputs and outputs that uses the `SoftMax()` function during training, the the one in the picture shown below...

<img src="./images/chapter_4_softmax.png" alt="a neural network with multiple inputs and outputs and softmax" style="width: 800px;">

 and the `ArgMax()` function during inference.

<img src="./images/chapter_4_argmax.png" alt="a neural network with multiple inputs and outputs and argmax" style="width: 800px;">


In this tutorial, you will...

- **[Import and Format Data and then Build a DataLoader From Scratch](#data)**
- **[Build a Neural Network with Multiple Inputs and Outputs and the SoftMax() Function](#build)**
- **[Train a Neural Network with Multiple Inputs and Outputs and the SoftMax() Function](#train)**
- **[Make Predictions with New Data Using the Trained Model and the ArgMax() Function](#predict)**

#### NOTE:
This tutorial assumes that you already know the basics of coding in **Python** and have read the first four chapters in **The StatQuest Illustrated Guide to Neural Networks and AI.**

-----

# Import the modules that will do all the work

The very first thing we need to do is load a bunch of Python modules. Python itself is just a basic programming language. These modules give us extra functionality to create and train a Neural Network.

In [ ]:
%%capture 
# %%capture prevents this cell from printing a ton of STDERR stuff to the screen

## First, check to see if lightning is installed, if not, install it.
import pip
try:
  __import__("lightning")
except ImportError:
  pip.main(['install', "lightning"])  

In [ ]:
import torch # torch will allow us to create tensors.
import torch.nn as nn # torch.nn allows us to create a neural network.
import torch.nn.functional as F # nn.functional give us access to the activation and loss functions.
from torch.optim import Adam # optim contains many optimizers. This time we're using Adam

import lightning as L # lightning has tons of cool tools that make neural networks easier
from torch.utils.data import TensorDataset, DataLoader # these are needed for the training data

import pandas as pd # We'll use pandas to read in the data and normalize it
from sklearn.model_selection import train_test_split # We'll use this to create training and testing datasets

----

<a id="data"></a>
# Import and Format data and then Build a DataLoader From Scratch.

Once we have the Python modules imported, now we need to import the data that we will use to train and test our neural network. Specifically, we're going to use the **[Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)**, which we will import from a comma-separated (CSV) text file so that we can learn how to build a DataLoader from scratch.

The **[Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)** is a classic dataset originally made famous by Rondal Fisher in 1936, and has since been used countless times to demonstrate the effectiveness of various classification algorithms. The dataset consists of 150 samples total, 50 for each of 3 species of Iris, **Setosa**, **Versicolor**, and **Virginica**. Each row in the dataset contains measurements for 4 variables: **[petal](https://en.wikipedia.org/wiki/Petal)** width and length and **[sepal](https://en.wikipedia.org/wiki/Sepal)** width and length.

**NOTE:** The data file we are going import, `iris.txt`, was originally downloaded from the **[UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/)**, which has a lot of great datasets that you can practice building Neural Networks (or any other machine learning algorithm) with. When you download the datasets from UCI, you get one file that has the data and another file that describes the data, including providing us with the names of each variable, or column, in the dataset. If you'd like to see the original data files, you can find them **[here](https://archive.ics.uci.edu/dataset/53/iris)**.

In [ ]:
## We'll read in the dataset with the pandas function read_table()
## read_table() can read in various text files including, comma-separated and tab-delimted.
df = pd.read_table('iris.txt', sep=",", header=None)
## NOTE: If the data were tab-delimted, we would set sep="\t".

Now, in theory, we have loaded the data into a DataFrame called `df`, but it's always a good idea to make sure this worked as expected. So, we'll print out the first handful of rows in the dataset with the `head()` method.

In [ ]:
## print out the first handful of rows using the head() method
df.head()

When we print out the first few rows of our new DataFrame, `df`, the first thing we see is that the columns are not named. In theory, it's fine to have unnamed columns (and just have numbers), but it makes the data hard to look at, so let's add the column names to `df`. To name each column, we simply assign a list of column names to `columns`.

In [ ]:
## To name each column, we assign a list of column names to `columns`
df.columns = ["sepal_length",
              "sepal_width", 
              "petal_length", 
              "petal_width", 
              "class"]

## To verify we did that correctly, let's print out the first few rows
df.head()

Hooray! Now that we can look at our DataFrame without getting a headache, let's see how big this dataset is and figure out how many different iris species we will have to train our neural network to predict. First, let's see how many rows and columns are in the dataset with `.shape`.

In [ ]:
df.shape ## shape returns the rows and colunns...

So, our dataset has 150 rows and 5 columns. Now let's see how many different types of iris are in it. We'll do this by counting the unique values in the column called `class` with `.nunique()`.

In [ ]:
## To determine the number of iris species in the dataset,
## we'll count the number of unique values in the column called `class`.
df['class'].nunique()

And we get the number we expected, 3. So that's good! Now let's print out the names of the 3 species with `.unique()`

In [ ]:
## We can print out the unique values in a dataframe's column with the 'unique()' method.
df['class'].unique()

So, just as we expected, we see that we have 3 different species of iris in our dataset: **Setosa**, **Versicolor**, **Virginica**.

Now let's verify that our dataset is balanced, meaning we have roughly the same number of entries (rows) in our data for each of the 3 iris species that we want our neural network to classify. We can do this with a fancy `for` loop that prints out the number of rows per class, regardless of the number of classes we have in our dataset.

In [ ]:
for class_name in df['class'].unique(): # for each unique class name...
    
    ## ...print out the number of rows associated with it
    print(class_name, ": ", sum(df['class'] == class_name), sep="") 

In this case, our dataset isn't just relatively well balanced, it is exactly balanced, and each class has exactly 50 rows of data associated with it. However, if things were really skewed, for example, we had 100 rows of data for **Setosa**, 100 rows for **Versicolor**, and only 10 rows of data for **Virginica**, then we might need to find some way to make the data more balanced. Balancing datasets is way out of the scope of this tutorial, but if you'd like to learn more with this simple **[Google search](https://www.google.com/search?q=how+to+balance+datasets)**.

Now, let's split the data into **training** and **testing** datasets. The first step is to separate the columns into input values and labels.

In this example, to keep the neural network simple, we'll just use `petal_width` and `sepal_width` values for the inputs. So the first we'll do is make sure we can correctly isolate the columns we want from the columns we don't want. We do this by passing `df` a list of column names we want to get values for, `['petal_width', 'sepal_width']`.

In [ ]:
## Print out the first few rows of just the `petal_width` and `sepal_width` columns
df[['petal_width', 'sepal_width']].head() 

Now that we have confirmed that we can correctly isolate the values for `petal_width` and `sepal_width`, let's use the original DataFrame, `df`, to create two new DataFrames. One DataFrame will have the petal and sepal widths, the values we will use to make predictions, and we'll call this DataFrame `input_values`. The other DataFrame will have the species, the values we will use to determine how good those predictions are, and this DataFrame will be called `label_values`.

In [ ]:
input_values = df[['petal_width', 'sepal_width']]
input_values.head()

In [ ]:
label_values = df['class']
label_values.head()

Now, because neural networks expect the inputs and output values to be numbers, we need to convert the values in the `label_values` into numbers, and we'll do this with `factorize()`.

In [ ]:
## Convert the strings in the 'class' column into numbers with factorize()
classes_as_numbers = label_values.factorize()[0] ## NOTE: factorize() returns a list of lists, 
                                                 ## and since we only need the first list of values,
                                                 ## we index the output of factorize() with [0].
classes_as_numbers ## print out the numbers

As we can see, the strings were converted into numbers. The first 50 values are 0, which represents **Setosa**. The following 50 values are 1, for **Versicolor**, and the last 50 values are 2, for **Viriginica**. 

Now, we need to split `input_values` and `classes_as_numbers` into **training** and **testing datasets**. And we do this with the **[sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)** function `train_test_split()`. **NOTE:** In practice, people usually use anywhere from 25-33% of the data for testing how well the model was trained. In this case, we'll use 25%, which is the default, but any percentage can be specified by setting the `test_size` parameter to a value between 0 and 1. Also, because we want to ensure that our test dataset has data for all three species of iris, we'll set `stratify=label_values`.

In [ ]:
input_train, input_test, label_train, label_test = train_test_split(input_values,
                                                                    classes_as_numbers, 
                                                                    test_size=0.25,
                                                                    stratify=classes_as_numbers)

Now we can verify that `train_test_split()` correctly put 75% of the data into `input_train` and `input_test` by printing out their shapes. Remember 75% of 150 = 112.5, so we would expect both `input_train` and `input_test` to have 112 rows.

In [ ]:
input_train.shape

In [ ]:
label_train.shape

Hooray!!! Both `input_train` and `label_train` have 112 rows, which is what we expect. Now, let's verify that the remaining 38 rows of data went into `input_test` and `label_test` by printing out their shapes.

In [ ]:
input_test.shape

In [ ]:
label_test.shape

Hooray again! `train_test_split()` did what we expected.

Now, because our neural network will have 3 outputs, one for each species (see the drawing of the neural network above), we need to convert the numbers in `label_train` into 3 element arrays, where each element in an array corresponds to a specific output in the neural network. Specifically, we'll use `[1.0, 0.0, 0.0]` to correspond to **Setosa**, `[0.0, 1.0, 0.0]` for  **Versicolor**, and `[0.0, 0.0, 1.0]` for **Virginica**. The good news is that we can easily do the **[one-hot encoding](https://youtu.be/589nCGeWG1w)**. We also tack on `type(torch.float32)` to ensure the numbers are saved in the correct format for the neural network to process efficiently.

In [ ]:
## Now create a new tensor with one-hot encoded rows for each row in the original dataset.
one_hot_label_train = F.one_hot(torch.tensor(label_train)).type(torch.float32)

**NOTE**: If we printed out the entire contents of `one_hot_label_train`, we'd get a matrix with 150 rows, which would take up a lot of space. So, instead, let's print out the first 10 rows.

In [ ]:
## Print out a few of the rows one-hot encoded data.
one_hot_label_train[:10]

So, as we can see in the output above, `classes_as_numbers` was correctly one-hot encoded and saved in `one_hot_label_train`.

Now, let's normalize the input variables so that their values range from 0 to 1. Normalizing data, so that it's all on the same scale, often makes it easier to train machine learning methods. In this case, since we have two datasets, `input_train` and `input_test`, we'll start determining the maximum and minimum values in `input_train`. Then we will use those values to normalize `input_train` and `input_test`. Using the maximum and minimum values from `input_train` to normalize both datasets avoids something called **Data Leakage**.

**NOTE:** If you don't know what it means to **normalize** your data, check out this **[short song](https://youtube.com/shorts/oZ9SrkF_-LE?feature=share)** that has a good beat, and you can dance to it.

In [ ]:
## First, determine the maximum values in input_train...
max_vals_in_input_train = input_train.max()
## Now print them out...
max_vals_in_input_train

In [ ]:
## Second, determine the minimum values in input_train
min_vals_in_input_train = input_train.min()
## Now print them out...
min_vals_in_input_train

In [ ]:
## Now normalize input_train with the maximum and minimum values from input_train
input_train = (input_train - min_vals_in_input_train) / (max_vals_in_input_train - min_vals_in_input_train)
input_train.head()

In [ ]:
## Now normalize input_test with the maximum and minimum values from input_train
input_test = (input_test - min_vals_in_input_train) / (max_vals_in_input_train - min_vals_in_input_train)
input_test.head()

Now, let's put our training data into a **DataLoader**, which we can use to train the neural network. **DataLoaders** are great for large datasets because they make it easy to access the data in batches, make it easy to shuffle the data each epoch, and they make it easy to use a relatively small fraction of the data if we want to do a quick and dirty training for debugging our code.

To put our data training data into a **DataLoader**, we'll start by converting `input_train` into tensors with `torch.tensor()`. We'll then combine `'input_train` with `one_hot_label_train` to create a **TensorDataset**. Lastly, we'll use the **TensorDataset** to create the **DataLoader**.

**NOTE:** `torch.tensor()` will get all bent out of shape if we pass it a DataFrame directly. So, instead of passing it a DataFrame, we pass it the values by tacking `.values` on to the end of each DataFrame. We also tack on `type(torch.float32)` to make sure the numbers are saved in the correct format for the neural network to process efficiently.

In [ ]:
## Convert the DataFrame input_train into tensors
input_train_tensors = torch.tensor(input_train.values).type(torch.float32)

## now print out the first 5 rows to make sure they are what we expect.
input_train_tensors[:5] 

**NOTE:** Because we'll also need to run `input_test` through the neural network, we'll need to convert it to tensors as well, and we might as well do it now.

In [ ]:
## Convert the DataFrame input_test into tensors
input_test_tensors = torch.tensor(input_test.values).type(torch.float32)

## now print out the first 5 rows to make sure they are what we expect.
input_test_tensors[:5] 

Now that we have tensors for `input_train`, named `input_train_tensors`, and we have the one-hot encoded `class` values stored in tensors called `label_train`, we can combine them into a **TensorDataset** that are, in turn, turned into **DataLoader**.

In [ ]:
train_dataset = TensorDataset(input_train_tensors, one_hot_label_train) 
train_dataloader = DataLoader(train_dataset)

# BAM!

At long last, we have created the **DataLoaders** that we need to train and test a neural network. Now, let's build the neural network.

----

<a id="build"></a>
# Building a neural network with multiple inputs and outputs with PyTorch and Lightning

Building a neural network with PyTorch means creating a new class. And to make it easy to train the neural network, this class will inherit from `LightningModule`.

Our new class will have the following methods:
- `__init__()` to initialize the Weights and Biases and keep track of a few other housekeeping things.
- `forward()` to make a forward pass through the neural network.
- `configure_optimizers()` to configure the optimizer. There are lots of optimizers to choose from, but in this tutorial, we'll change things up and use `Adam`.
- `training_step()` to pass the training data to `forward()`, calculate the loss and keep track of the loss values in a log file.

Also, for reference, here is a picture of the neural network we want to create:
<img src="./images/chapter_4_softmax.png" alt="a neural network with multiple inputs and outputs and softmax" style="width: 800px;">

As we can see in the picture, our neural network has 2 inputs, one for Petal Width and one for Sepal Width, a single hidden layer with two **[ReLU](https://youtu.be/68BZ5f7P94E)** activation functions, and 3 outputs, one for each species of iris.

So, given this specification for this neural network, let's code it in a new class called `MultipleInsOuts`.

In [ ]:
class MultipleInsOuts(L.LightningModule):
    
    def __init__(self):
        
        super().__init__() ## We call the __init__() for the parent, LightningModule, so that it
                           ## can initialize itself as well.
        
        ## Now we the seed for the random number generorator.
        ## This ensures that when you create a model from this class, that model
        ## will start off with the exact same random numbers that I started out with when
        ## I created this demo. At least, I hope that is what happens!!! :)
        L.seed_everything(seed=42)
        
        ############################################################################
        ##
        ## Here is where we initialize the Weights and Biases for the neural network
        ##
        ############################################################################
        
        ## If you look at the drawing of the network we want to build (above), 
        ## you see that we have 2 inputs that lead to 2 activation functions.
        ## We create these connections and initialize their Weights and Biases
        ## with the nn.Linear() function by setting in_features=2 and out_features=2.
        self.input_to_hidden = nn.Linear(in_features=2, out_features=2, bias=True)
        
        ## Next, we see that the 2 activation functions are connected to 3 outputs.
        ## We create these connections and initialize their Weights and Biases
        ## with the nn.Linear() function by setting in_features=2 and out_features=3. 
        self.hidden_to_output = nn.Linear(in_features=2, out_features=3, bias=True)
        
        ## We'll use Cross Entropy to calculate the loss between what the 
        ## neural network's predictions and actual, or known, species for
        ## each row in the dataset.
        ## To learn more about Cross Entropy, see: https://youtu.be/6ArSys5qHAU
        ## NOTE: nn.CrossEntropyLoss applies a SoftMax function to the values
        ## we give it, so we don't have to do that oursevles. However,
        ## when we use this neural network (after it has been trained), we'll
        ## have to remember to apply a SoftMax function to the output.
        self.loss = nn.CrossEntropyLoss()
    
    
    def forward(self, input): 
        ## First, we run the input values to the activation functions
        ## in the hidden layer
        hidden = self.input_to_hidden(input)
        ## Then we run the values through a ReLU activation function
        ## and then run those values to the output.
        output_values = self.hidden_to_output(torch.relu(hidden))
            
        return(output_values)
        
    
    def configure_optimizers(self):
        ## In this example, configuring the optimizer
        ## consists of passing it the weights and biases we want
        ## to optimize, which are all in self.parameters(),
        ## and setting the learning rate with lr=0.001.
        return Adam(self.parameters(), lr=0.001)

    
    def training_step(self, batch, batch_idx):
        ## The first thing we do is split 'batch'
        ## into the input and label values.
        inputs, labels = batch 
        
        ## Then we run the input through the neural network
        outputs = self.forward(inputs)
        
        ## Then we calculate the loss.
        loss = self.loss(outputs, labels)
        
        ## Lastly, we could add the loss a log file
        ## so that we can graph it later. This would
        ## help us decide if we have done enough training
        ## Ideally, if we do enough training, the loss
        ## should be small and not getting any smaller.
        # self.log("loss", loss) 
        
        return loss

Now that we've created a class for our neural network, let's train it.

----

<a id="train"></a>
# Training our Neural Network

Training our new neural network means we create a model from the new class, `MultipleInsOuts`...

In [ ]:
model = MultipleInsOuts()

...and then create a **Lightning Trainer**, `L.Trainer`, and use it to optimize the parameters. **NOTE:** We will start with 10 epochs, complete runs through our training data. This may be enough to successfully optimize all of the parameters, but it might not. We'll find out later in the tutorial when we make a graph of how the loss values change during training.

In [ ]:
trainer = L.Trainer(max_epochs=10)
trainer.fit(model, train_dataloaders=train_dataloader)

Hooray! We've trained the model with 10 epochs! Now, let's see if the predictions are any good. We can do this by seeing how well it predicts the testing data. We'll start by running `input_test_tensors` through the neural network and saving the output `predictions`.

In [ ]:
# Run the input_test_tensors through the neural network
predictions = model(input_test_tensors)

Now, because our neural network has three outputs, one for **Setosa**, one for **Versicolor**, and one for **Virginica**, we should get 3 values for each row in `input_test_tensors`. We can verify that by looking at the first few rows of `predictions`.

In [ ]:
predictions[0:4,]

We can determine which species was predicted in `predictions` by selecting the index in each row that corresponding to the largest value, and we do that with `torch.argmax()`. `torch.argmax()` returns a tensor that contains the indices with the largest values for each row.

In [ ]:
## Select the output with highest value...
predicted_labels = torch.argmax(predictions, dim=1) ## dim=0 applies argmax to rows, dim=1 applies argmax to columns
predicted_labels[0:4] # print out the first 4 predictions

In the first row index 0 had the largest value. Thus, the first prediction corresponds to **Setosa**. The second, third, and fourth rows predicted 2, which corresponds to **Virginica**.

Now, let's compare what the neural network predicted in `predicted_labels` to the known values in `label_test` and calculate the percentage of correct predictions. We do this by adding up the number of times an element in `predicted_labels` equals the corresponding element in `label_test` and dividing by the number of elements in `predicted_labels`.

In [ ]:
## Now compare predicted_labels with test_labels to calculate accuracy
## NOTE: torch.eq() computes element-wise equality between two tensors.
##       label_test, however, is just an array, so we convert it to a tensor
##       before passing it in. torch.sum() then adds up all of the "True"
##       output values to get the number of correct predictions. 
##       We then divide the number of correct predictions by the number of predicted values,
##       obtained with len(predicted_labels), to get the percentage of correct predictions
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

And we see that our neural network only correctly predicts 66% of the testing data. This isn't very good. So, will training our model for more epochs improve the model's predictions?

One way to answer that question is to just train for longer and see what happens.

The good news is that because we're using **Lightning**, we can pick up where we left off training without starting over from scratch. This is because training with **Lightning** creates _checkpoint_ files that keep track of the Weights and Biases as they change. As a result, all we have to do to pick up where we left off is tell the `Trainer` where the checkpoint files are. This is awesome and will save us a lot of time since we don't have to retrain the first **10** epochs. So, let's add an additional **90** epochs to the training.

To add additional epochs to the training, we first identify where the checkpoint file is with the following command.

In [ ]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path ## By default, "best" = "most recent"

Then we create a new Lightning Trainer, just like before, but we set the number of epochs to 100. Given that we already trained for 10 epochs, this means we'll do 90 more.

In [ ]:
## First, create a new Lightning Trainer
trainer = L.Trainer(max_epochs=100) # Before, max_epochs=10, so, by setting it to 100, we're adding 90 more.

## Then call trainer.fit() using the path to the most recent checkpoint files
## so that we can pick up where we left off.
trainer.fit(model, train_dataloaders=train_dataloader, ckpt_path=path_to_checkpoint)

Now, let's run the testing data through the network and calculate the accuracy. We'll do this just like we did before.

In [ ]:
# Run the input_test_tensors through the neural network
predictions = model(input_test_tensors)

## Select the output with highest value...
predicted_labels = torch.argmax(predictions, dim=1) ## dim=0 applies softmax to rows, dim=1 applies softmax to columns

## Now compare predicted_labels with test_labels to calculate accuracy
## NOTE: torch.eq() computes element-wise equality between two tensors.
##       label_test, however, is just an array, so we convert it to a tensor
##       before passing it in. torch.sum() then adds up all of the "True"
##       output values to get the number of correct predictions. 
##       We then divide the number of correct predictions by the number of predicted values,
##       obtained with len(predicted_labels), to get the percentage of correct predictions
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

After 100 training epochs, we correctly classified 95% of the testing data.

# Double BAM!!

In [ ]:
## Print out the name and value for each named parameter
## parameter in the model. Remember parameters are variables,
## like Weights and Biases, that we can train.
for name, param in model.named_parameters():
    print(name, torch.round(param.data, decimals=2))

----

<a id="predict"></a>
# Make a Prediction with New Data

Now that our model is trained, we can use it to make predictions from new data. This is done by passing the model a tensor with normalized petal and sepal widths wrapped up in a tensor, and then passing the model output to `torch.argmax()`.

For example, if the raw petal and sepal width measurements were 0.2 and 3.0, we would first normalize them using the maximum and minimum values we calculated with the training data.

In [ ]:
normalized_values = ([0.2, 3.0] - min_vals_in_input_train) / (max_vals_in_input_train - min_vals_in_input_train)
normalized_values

Then we convert `normalized_values` into a tensor and pass it to the model, and pass the output to the `argmax()` function

In [ ]:
torch.argmax(model(torch.tensor(normalized_values).type(torch.float32)))

And the output is 0, meaning that the neural network predicts that the measurements come from **Setosa**.

# TRIPLE BAM!!!